In [1]:
import pandas as pd
from sodapy import Socrata
import numpy as np 
from pyspark.sql import SparkSession 
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql import Row 
from pyspark.sql.functions import avg 
import os 
from pyspark.ml.feature import Imputer
from pyspark.sql.functions import col

In [2]:
# Using sodapy get data from dataset
client = Socrata(
	"data.seattle.gov", 
	"sbICfPVtvsiIsfUuzOu7houHc", 
	"maluzhkovitch@gmail.com",
	"Gt#103277")

results = client.get("hiyf-7edq", limit=2000)

In [3]:
# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)


In [4]:
results_df.describe()


,occupancydatetime,paidoccupancy,blockfacename,sideofstreet,sourceelementkey,parkingtimelimitcategory,parkingspacecount,paidparkingarea,paidparkingsubarea,parkingcategory,location,:@computed_region_ru88_fbhk,:@computed_region_kuhn_3gp2,:@computed_region_q256_3sug
count,1000,1000,1000,1000,1000,999,1000,1000,725,1000,1000,1000,1000,1000
unique,690,22,573,8,703,4,31,20,7,3,703,15,22,13
top,2022-09-03T17:32:00.000,1,E PINE ST BETWEEN BOYLSTON AVE AND HARVARD AVE,W,14697,120,4,Belltown,North,Paid Parking,"{'type': 'Point', 'coordinates': [-122.3227780...",56,9,19576
freq,5,208,7,182,6,589,138,202,211,946,6,222,143,212


In [5]:
results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 14 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   occupancydatetime            1000 non-null   object
 1   paidoccupancy                1000 non-null   object
 2   blockfacename                1000 non-null   object
 3   sideofstreet                 1000 non-null   object
 4   sourceelementkey             1000 non-null   object
 5   parkingtimelimitcategory     999 non-null    object
 6   parkingspacecount            1000 non-null   object
 7   paidparkingarea              1000 non-null   object
 8   paidparkingsubarea           725 non-null    object
 9   parkingcategory              1000 non-null   object
 10  location                     1000 non-null   object
 11  :@computed_region_ru88_fbhk  1000 non-null   object
 12  :@computed_region_kuhn_3gp2  1000 non-null   object
 13  :@computed_region_q256_3sug  1000 

In [6]:
# Removing additional pre calculated columns
results_df.drop(columns=[':@computed_region_ru88_fbhk', ':@computed_region_kuhn_3gp2', ':@computed_region_q256_3sug'], axis=1, inplace=True)
results_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   occupancydatetime         1000 non-null   object
 1   paidoccupancy             1000 non-null   object
 2   blockfacename             1000 non-null   object
 3   sideofstreet              1000 non-null   object
 4   sourceelementkey          1000 non-null   object
 5   parkingtimelimitcategory  999 non-null    object
 6   parkingspacecount         1000 non-null   object
 7   paidparkingarea           1000 non-null   object
 8   paidparkingsubarea        725 non-null    object
 9   parkingcategory           1000 non-null   object
 10  location                  1000 non-null   object
dtypes: object(11)
memory usage: 86.1+ KB


# Spark part start here

In [7]:
spark = SparkSession.builder.appName('Paid_Parking_Occupancy').getOrCreate()

In [8]:
sparkDF=spark.createDataFrame(results_df.astype(str))

In [9]:
sparkDF.printSchema()


root
 |-- occupancydatetime: string (nullable = true)
 |-- paidoccupancy: string (nullable = true)
 |-- blockfacename: string (nullable = true)
 |-- sideofstreet: string (nullable = true)
 |-- sourceelementkey: string (nullable = true)
 |-- parkingtimelimitcategory: string (nullable = true)
 |-- parkingspacecount: string (nullable = true)
 |-- paidparkingarea: string (nullable = true)
 |-- paidparkingsubarea: string (nullable = true)
 |-- parkingcategory: string (nullable = true)
 |-- location: string (nullable = true)



In [10]:
sparkDF.show(n=10)

Py4JJavaError: An error occurred while calling o44.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0) (DESKTOP-J9G4M0M executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:189)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:164)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:65)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:365)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:329)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoin